In [ ]:
!pip install arch
#!pip install spa
import yfinance as yf
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import arch
from arch import arch_model
import matplotlib.pyplot as plt
from arch.bootstrap import SPA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 10.7 MB/s eta 0:00:00


In [ ]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ECMA 31330 Final Project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

Mounted at /content/drive
['Articles', 'SentimentScores', 'Aakash Gupta Final Project Presentation.gslides', 'Sp500Stocks.csv', 'SPY2015.csv', 'SPY2020.csv', 'SPY-2015_2020.csv', 'SentimentAnalysisPipeline.ipynb', 'SimpleGARCHModel.ipynb', 'LassoModel.ipynb', 'NeuralNet.ipynb', 'DataCleaningPipeline.ipynb', 'GARCHFittingPipeline.ipynb']


In [ ]:
targeted_path = GOOGLE_DRIVE_PATH + "/SentimentScores"
csv_files = [file for file in os.listdir(targeted_path) if file.endswith('.csv')]
dataframes = []
for file in csv_files:
    file_path = os.path.join(targeted_path, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)



In [ ]:
sentiment_df = combined_df[['date', 'Positive', 'Negative', 'Neutral']].groupby('date').quantile(0.90).reset_index()
sentiment_df

,date,Positive,Negative,Neutral
0,1975-01-01,0.087603,0.597122,0.927007
1,2015-01-01,0.028760,0.656124,0.315116
2,2015-01-02,0.057416,0.849696,0.503523
3,2015-01-03,0.084118,0.859030,0.894526
4,2015-01-04,0.239992,0.883377,0.895273
...,...,...,...,...
1945,2020-12-27,0.196491,0.805055,0.936801
1946,2020-12-28,0.066696,0.775040,0.927744
1947,2020-12-29,0.139268,0.722470,0.922212
1948,2020-12-30,0.109317,0.739341,0.926059


In [ ]:
stocks = pd.read_csv(GOOGLE_DRIVE_PATH + "/SPY-2015_2020.csv")
stocks = stocks[2:]
stocks["Return"] = 100 * pd.to_numeric(stocks.Close).pct_change()
stocks["Volatility"] = abs((pd.to_numeric(stocks.Open) - pd.to_numeric(stocks.Close))/pd.to_numeric(stocks.Open))
stocks["date"] = stocks['Price']
#stocks2015["date"] = pd.to_datetime(stocks2015['Price'], format = '%Y-%m-%d')

In [ ]:
stocks

,Price,Close,High,Low,Open,Volume,Return,Volatility,date
2,2014-12-31,172.68533325195312,174.91175014913608,172.55931521728235,174.74372182966204,130333800,NaN,0.011779,2014-12-31
3,2015-01-02,172.59291076660156,173.8111448920031,171.54271777555792,173.39106769558566,121465900,-0.053521,0.004603,2015-01-02
4,2015-01-05,169.4759063720703,171.70230987578518,169.16505342016106,171.5342815996109,169632600,-1.805986,0.012000,2015-01-05
5,2015-01-06,167.87962341308594,170.3160655379834,167.0730697163054,169.78676427385443,209151400,-0.941894,0.011233,2015-01-06
6,2015-01-07,169.97158813476562,170.31605442130996,168.77017382476237,169.2238514360209,125346700,1.246110,0.004419,2015-01-07
...,...,...,...,...,...,...,...,...,...
1508,2020-12-23,347.1472473144531,349.08333460443055,346.81668809319564,347.8177905425075,46201400,0.089880,0.001928,2020-12-23
1509,2020-12-24,348.4977111816406,348.5260431760125,347.03384343581655,347.6288153176249,26457900,0.389018,0.002499,2020-12-24
1510,2020-12-28,351.4916687011719,351.88831670770304,350.45278064744303,351.08553826518647,39000400,0.859104,0.001157,2020-12-28
1511,2020-12-29,350.8210754394531,353.2199570144629,350.2260746436619,353.0405114150347,53680500,-0.190785,0.006287,2020-12-29


In [ ]:
final_df = pd.merge(sentiment_df, stocks, on = 'date', how = 'inner')
final_df_filtered = final_df[["Negative", "Return", 'date', "Volatility"]].dropna()

In [ ]:
np.corrcoef(final_df_filtered["Negative"], final_df_filtered["Volatility"])

array([[1.        , 0.03368022],
       [0.03368022, 1.        ]])

In [ ]:
training_data = final_df_filtered[:-300]
testing_data = final_df_filtered[-300:]

In [ ]:
baseline_model = arch_model(training_data['Return'], p = 2, q = 1).fit()
exog_model = arch_model(training_data['Return'], p = 2, q = 1, x = training_data['Negative'], mean = "ARX").fit()

Iteration:      1,   Func. Count:      7,   Neg. LLF: 12191.458685762707
Iteration:      2,   Func. Count:     17,   Neg. LLF: 3739489.5908708177
Iteration:      3,   Func. Count:     24,   Neg. LLF: 1225.6920612854487
Iteration:      4,   Func. Count:     31,   Neg. LLF: 3488.1412042441852
Iteration:      5,   Func. Count:     39,   Neg. LLF: 1173.6096443423194
Iteration:      6,   Func. Count:     46,   Neg. LLF: 521967833.00596344
Iteration:      7,   Func. Count:     53,   Neg. LLF: 1428.199089681686
Iteration:      8,   Func. Count:     61,   Neg. LLF: 1169.6764688353578
Iteration:      9,   Func. Count:     67,   Neg. LLF: 1169.6878157280298
Iteration:     10,   Func. Count:     74,   Neg. LLF: 1169.6689397671576
Iteration:     11,   Func. Count:     81,   Neg. LLF: 1169.668733817657
Iteration:     12,   Func. Count:     86,   Neg. LLF: 1169.6687338176853
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1169.668733817657
            Iterat

In [ ]:
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 Return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1169.67
Distribution:                  Normal   AIC:                           2349.34
Method:            Maximum Likelihood   BIC:                           2374.16
                                        No. Observations:                 1058
Date:                Thu, Mar 06 2025   Df Residuals:                     1057
Time:                        22:19:37   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0893  2.019e-02      4.421  9.818e-06 [4.969e-02,  0.129]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0300  1.642e-02      1.825  6.798e-02 [-2.214e-03,6.216e-02]
alpha[1]       0.1701  5.209e-02      3.265  1.094e-03    [6.799e-02,  0.272]
alpha[2]       0.0000  8.386e-02      0.000      1.000      [ -0.164,  0.164]
beta[1]        0.7904  7.916e-02      9.985  1.779e-23      [  0.635,  0.946]
=============================================================================

Covariance estimator: robust
"""

In [ ]:
exog_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          AR-X - GARCH Model Results                          
==============================================================================
Dep. Variable:                 Return   R-squared:                      -0.003
Mean Model:                      AR-X   Adj. R-squared:                 -0.004
Vol Model:                      GARCH   Log-Likelihood:               -1169.50
Distribution:                  Normal   AIC:                           2350.99
Method:            Maximum Likelihood   BIC:                           2380.77
                                        No. Observations:                 1058
Date:                Thu, Mar 06 2025   Df Residuals:                     1056
Time:                        22:19:37   Df Model:                            2
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
Const          0.1062  3.265e-02      3.253  1.142e-03 [4.222e-02,  0.170]
Negative      -0.0401  7.633e-02     -0.525      0.600   [ -0.190,  0.110]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0299  1.658e-02      1.801  7.168e-02 [-2.633e-03,6.236e-02]
alpha[1]       0.1681  5.238e-02      3.208  1.336e-03    [6.539e-02,  0.271]
alpha[2]   3.3264e-16  8.334e-02  3.991e-15      1.000      [ -0.163,  0.163]
beta[1]        0.7922  7.985e-02      9.921  3.375e-23      [  0.636,  0.949]
=============================================================================

Covariance estimator: robust
"""

In [ ]:
#We will do a one year forecast
train = final_df_filtered[:-300]
test = final_df_filtered[-300:]
baseline_forecast = baseline_model.forecast(start = train['Return'], horizon = len(test))
exog_forecast = exog_model.forecast(start = train['Return'], x = test['Negative'], horizon = len(test))

actual_vol = np.array(test['Volatility'])

baseline_mse = sum(baseline_forecast.variance.iloc[-1:].values[0] - np.array(test["Volatility"]))**2/300
exog_mse = sum(exog_forecast.variance.iloc[-1:].values[0] - np.array(test["Volatility"]))**2/300


In [ ]:
baseline_mse

154.05344532305642

In [ ]:
exog_mse

151.2363246499778